# 📘 Nuggetizer: A lightweight nugget-based evaluation framework for pyterrier-rag

## 📌 Introduction
- Objective: Demonstrate how to use NuggetizerRAG, a personal library for nugget-based evaluation in Retrieval-Augmented Generation (RAG).
- Context: Inspired by the AutoNuggetizer framework from the TREC 2024 RAG Track.
- Use case: Provide interpretable and automatic evaluation metrics for open-domain QA and generation tasks using PyTerrier pipelines.

## 🎯 Motivation and Background
- The Problem: Traditional RAG evaluations rely on lexical overlap or ROUGE scores, which miss semantic correctness.
- The Solution: Nugget evaluation, originally proposed in TREC QA 2003, revived by AutoNuggetizer, uses semantically atomic facts (“nuggets”) to evaluate answers.
- Inspiration: This library reimplements a simplified, local version of AutoNuggetizer with modular hooks into PyTerrier and HuggingFace models.

## ⚙️ Installation and Setup

In [1]:
!pip install git+https://github.com/MattiWe/ir_datasets.git@add-msmarco-v2.1-trec-rag

  Cloning https://github.com/MattiWe/ir_datasets.git (to revision add-msmarco-v2.1-trec-rag) to /tmp/pip-req-build-btwtay6w
  Running command git clone --filter=blob:none --quiet https://github.com/MattiWe/ir_datasets.git /tmp/pip-req-build-btwtay6w
  Running command git checkout -b add-msmarco-v2.1-trec-rag --track origin/add-msmarco-v2.1-trec-rag
  Switched to a new branch 'add-msmarco-v2.1-trec-rag'
  Branch 'add-msmarco-v2.1-trec-rag' set up to track remote branch 'add-msmarco-v2.1-trec-rag' from 'origin'.
  Resolved https://github.com/MattiWe/ir_datasets.git to commit 24a983d51b04f11a11c2f654dab3c275905c67a0
  Preparing metadata (setup.py) ... done


In [2]:
!pip install -q python-terrier pyterrier_t5 pyterrier_pisa

In [3]:
!pip install -q git+https://github.com/terrierteam/pyterrier_rag.git

In [4]:
!pip install -q --no-deps ../.

In [5]:
import pyterrier as pt
from pyterrier_rag.backend import Backend

/opt/miniconda3/envs/nuggetizer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [6]:
import ir_datasets
dataset = ir_datasets.load('msmarco-segment-v2.1')

In [7]:
pt_dataset = pt.get_dataset("irds:msmarco-segment-v2.1")

# Pipelines

In [8]:
from pyterrier_pisa import PisaIndex
from pyterrier_t5 import MonoT5ReRanker

def rename_segment(run):
    run = run.rename(columns={"segment": "text"})
    return run
rename_pipe = pt.apply.generic(rename_segment)

index = PisaIndex('/mnt/indices/msmarco-segment-v2.1.pisa/')
bm25_ret = index.bm25() >> pt.text.get_text(pt_dataset, "segment") >> rename_pipe
monoT5 = MonoT5ReRanker(batch_size=64, verbose=False)
monoT5_ret = bm25_ret % 10 >> monoT5

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# Building a baseline retrieval run to generate baseline nuggets

In [9]:
import pandas as pd
df = pd.read_csv("../../diversification/diversy-rag/datasets/TREC-RAGgy/raggy-dev.tsv", sep="\t", dtype=str)
topics_df  = df[["qid", "query"]]
answers_df = df[["qid", "query", "gold_answer"]]

In [10]:
bm25_ret.search("hello world")

,qid,query,docno,score,rank,text
0,1,hello world,msmarco_v2.1_doc_30_206096153#9_462542248,15.437883,0,"However, if you’re looking for something a bit..."
1,1,hello world,msmarco_v2.1_doc_30_206096153#10_462544673,15.426567,1,"In other words, I explore languages almost ran..."
2,1,hello world,msmarco_v2.1_doc_56_166221946#0_369357076,15.149368,2,Hello World Program in Java\n\n\n\n\n\n\n\nRel...
3,1,hello world,msmarco_v2.1_doc_24_1026769510#2_2162703600,15.075923,3,"1\n""Hello World"" -ieq ""hello world""\nEqual Che..."
4,1,hello world,msmarco_v2.1_doc_01_1568473385#1_2272488579,15.056515,4,nmake | msbuild\n\nmake\nrake\nant\ngradle\nve...
...,...,...,...,...,...,...
995,1,hello world,msmarco_v2.1_doc_28_1238377198#4_2648854685,13.674044,995,Share\nedited Dec 3 '19 at 16:23\nVadim Shkabe...
996,1,hello world,msmarco_v2.1_doc_09_332951182#16_455124934,13.672531,996,"WAIT KEY\nPBasic\nDEBUG ""Hello, World!"", CR\no..."
997,1,hello world,msmarco_v2.1_doc_25_897484862#11_1695398270,13.672531,997,If you don't already have Cargo installed on y...
998,1,hello world,msmarco_v2.1_doc_51_1184196669#3_2398216740,13.672531,998,It does nothing. It takes the decorated functi...


In [11]:
baseline = (monoT5_ret)(topics_df.head(10))
baseline

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


,qid,query,docno,text,score,rank
0,23287,are landlords liable if someone breaks in a hu...,msmarco_v2.1_doc_02_759557285#0_1325339642,Is a landlord liable if a tenant or visitor is...,-0.002110,0
1,23287,are landlords liable if someone breaks in a hu...,msmarco_v2.1_doc_48_841527758#3_1529122925,"1996), reh'g denied (1996).) If a landlord is ...",-0.035341,6
2,23287,are landlords liable if someone breaks in a hu...,msmarco_v2.1_doc_02_759557285#1_1325342568,"To do this, the injured person must show that:...",-0.003268,1
3,23287,are landlords liable if someone breaks in a hu...,msmarco_v2.1_doc_48_841527758#12_1529136555,But if the tenant has a month-to-month rental ...,-0.027581,5
4,23287,are landlords liable if someone breaks in a hu...,msmarco_v2.1_doc_48_841527758#11_1529134815,most courts hold landlords liable for knowing ...,-0.065491,8
...,...,...,...,...,...,...
95,395948,information about who howard gardner and what ...,msmarco_v2.1_doc_43_701261032#3_1477785431,Multiple Intelligences Test\nBased on the work...,-4.660836,8
96,395948,information about who howard gardner and what ...,msmarco_v2.1_doc_45_911384240#2_1740648328,"He is the director of Harvard Project Zero , A...",-0.016688,2
97,395948,information about who howard gardner and what ...,msmarco_v2.1_doc_01_1630976798#1_2378503407,and some of the issues around its conceptualiz...,-0.748225,6
98,395948,information about who howard gardner and what ...,msmarco_v2.1_doc_44_376498252#0_936606965,Multiple Intelligences (Howard Gardner) - Inst...,-5.776029,9


# Nuggetizer setup

In [12]:
from pyterrier_rag.backend import HuggingFaceBackend

backend =  HuggingFaceBackend("hugging-quants/Meta-Llama-3.1-70B-Instruct-AWQ-INT4",
                                          max_new_tokens=2048,
                                          model_args={
                                              "device_map": "cuda"
                                          }
                                         )

Loading checkpoint shards: 100%|██████████| 9/9 [00:05<00:00,  1.73it/s]


In [13]:
from fastchat.conversation import register_conv_template, get_conv_template, Conversation, SeparatorStyle

register_conv_template(
    Conversation(
        name="meta-llama-3.1-sp",
        system_message="",
        roles=("user", "assistant"),
        sep_style=SeparatorStyle.ADD_COLON_SINGLE,
        sep="\n",
        messages=[],
    )
)

conv_template = get_conv_template("meta-llama-3.1-sp")

In [14]:
from open_nuggetizer.nuggetizer import Nuggetizer

nuggetizer = Nuggetizer(
    backend=backend, 
    conversation_template=conv_template,
    verbose=True
)
nuggets = nuggetizer.create(baseline)
nuggetizer.score(nuggets)

  0%|          | 0/1 [00:00<?, ?window/s]/opt/miniconda3/envs/nuggetizer/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


PROMPT:
 ['You are NuggetizeLLM, an intelligent assistant that can update a list of atomic nuggets to best provide all the information required for the query.\nuser: Update the list of atomic nuggets of information (1-12 words), if needed, so they best provide the information required for the query. Leverage only the initial list of nuggets (if exists) and the provided context (this is an iterative process).  Return only the final list of all nuggets in a Pythonic list format (even if no updates). Make sure there is no redundant information. Ensure the updated nugget list has at most 30 nuggets (can be less), keeping only the most vital ones. Order them in decreasing order of importance. Prefer nuggets that provide more interesting information.\n\nSearch Query: are landlords liable if someone breaks in a hurts tenant\nContext:\n[1] Is a landlord liable if a tenant or visitor is injured on the rental property? | OpenJurist\nIs a landlord liable if a tenant or visitor is injured on the r

From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.

pt.apply.by_query():  11%|█         | 11/100 [00:29<03:57,  2.67s/it]

--- RAW TEXT: ---
 ["Landlord liable for injuries on rental property", "Tenant must prove landlord negligence", "Landlord must maintain common areas", "Tenant can sue for medical bills and lost earnings", "Landlord liable for injuries caused by tenant's dog", "Landlord must know dog is dangerous", "Landlord must have power to remove dog", "Landlord liable for injuries off rental property", "Landlord must maintain property in safe condition", "Landlord liable for faulty wiring and toxic mold", "Landlord not liable for conditions arising after tenant takes possession", "Landlord can be held liable for tenant's behavior if aware and does nothing"] 
 --- END OF RAW TEXT---



  0%|          | 0/1 [00:00<?, ?window/s]

...

  0%|          | 0/2 [00:00<?, ?window/s]

PROMPT:
 ['You are NuggetizeScoreLLM, an intelligent assistant that can label a list of atomic nuggets based on their importance for a given search query.\nuser: Based on the query, label each of the 681 nuggets either a vital or okay based on the following criteria. Vital nuggets represent concepts that must be present in a “good” answer; on the other hand, okay nuggets contribute worthwhile information about the target but are not essential. Return the list of labels in a Pythonic list format (type: List[str]). The list should be in the same order as the input nuggets. Make sure to provide a label for each nugget.\n\nSearch Query: information about who howard gardner and what does he do\nNugget List: [1] Howard Gardner is an American psychologist\n[2] He specializes in cognitive and developmental psychology\n[3] He is best known for his theory of multiple intelligences\n[4] Gardner believes that the way people usually think about intelligence is too narrow\n[5] He was born on July 11

100%|██████████| 2/2 [00:08<00:00,  4.05s/window]

--- RAW TEXT: ---
 ['vital', 'vital', 'vital', 'vital', 'okay', 'okay', 'okay', 'vital', 'vital', 'vital'] 
 --- END OF RAW TEXT---


,qid,query,nugget_id,nugget,importance
0,23287,are landlords liable if someone breaks in a hu...,23287_1,Landlord liable for injuries on rental property,1
1,23287,are landlords liable if someone breaks in a hu...,23287_2,Tenant must prove landlord negligence,1
2,23287,are landlords liable if someone breaks in a hu...,23287_3,Landlord must maintain common areas,1
3,23287,are landlords liable if someone breaks in a hu...,23287_4,Tenant can sue for medical bills and lost earn...,0
4,23287,are landlords liable if someone breaks in a hu...,23287_5,Landlord liable for injuries caused by tenant'...,0
...,...,...,...,...,...
95,395948,information about who howard gardner and what ...,395948_6,His parents were Ralph and Hilde Gardner,0
96,395948,information about who howard gardner and what ...,395948_7,They were German-Jewish immigrants who fled fr...,0
97,395948,information about who howard gardner and what ...,395948_8,Gardner is the John H. and Elisabeth A. Hobbs ...,1
98,395948,information about who howard gardner and what ...,395948_9,He is the senior director of Harvard Project Zero,1


# Evaluation

In [15]:
results = pt.Experiment(
    [
        load_baseline, 
    ],
    topics_df, 
    answers_df,
    [pyterrier_rag.measures.F1, nuggetizer.VitalScore(), nuggetizer.StrictVitalScore()],
    #batch_size=25,
    names=['baseline retriever'],
)

NameError: name 'load_baseline' is not defined